# Exercícios: Retail Sales

__Exercício:__ o arquivo `sales_data_full.csv` contém o histórico de vendas de 2018 de uma grande rede varejista. O arquivo está dividido da seguinte forma:

`product_id|store_id|[date list]|[quantity list]|[price list]`

Hoje o diretor de vendas gostaria de rever o portfolio das lojas e para isso gostaria de entender:
1. Qual foi a venda com maior preço unitário? 
2. Quando essa venda aconteceu?
3. Qual foi a quantidade vendida?
4. Quais foram as cinco vendas com maior preço unitário?

Além disso responda:
1. Quantas partições o spark utilizou para quebrar seu problema?
2. Qual foi o tempo de execução do exercício?
3. Quantos processadores foram utilizados pelo Spark?

Vamos inicialmente importar a biblioteca de expressões regulares a ser utilizada posteriormente:

In [1]:
import re

# Exercícios 01

* Varie o número de cores para 2, 4 e 6 e calcule o tempo de execução. Verifique no Application UI e nos recursos da máquina a uitilização dos porcessadores;
* Verifique o DAG do problema de sua resolução.

Lembre-se de terminar a sessão antes de iniciar uma nova.

Para o caso do varejo 2018, encontre:
1. A venda de maior quantidade;
2. O número de vendas no ano;
3. A quantidade total vendida no ano;
4. A receita total vendida no ano por loja.

In [2]:
import datetime as dt

start = dt.datetime.now()

In [3]:
# importa a biblioteca do spark
from pyspark.sql import SparkSession

In [4]:
# cria a sessão do spark
spark = SparkSession.builder \
    .master('local[6]') \
    .appName('Exercicio_Helena')\
    .getOrCreate() 

In [5]:
spark

In [6]:
# acessa o contexto do spark, no qual as funções de RDDs estão disponíveis
sc = spark.sparkContext

In [7]:
datapath = '../data/retail_data'

In [8]:
datafile = datapath + '/sales_data_2018.csv'

In [9]:
rdd = sc.textFile(datafile)

In [10]:
from support_functions import trata_rdd

In [11]:
dados_tratados = rdd.map(lambda el: trata_rdd(el))

In [12]:
dados_tratados.take(2)

[[1, 33, ['2018-03-02', '2018-09-05'], [1, 1], [80.44, 104.71]],
 [2, 10, ['2018-04-13'], [1], [99.29]]]

## Venda com maior quantidade

In [13]:
qtd_max = dados_tratados.map(lambda el: el + [max(el[-2])])

In [14]:
qtd_max.take(2)

[[1, 33, ['2018-03-02', '2018-09-05'], [1, 1], [80.44, 104.71], 1],
 [2, 10, ['2018-04-13'], [1], [99.29], 1]]

In [15]:
#reduce compara 2 elementos da RDD
#deixa apenas 1 elemento de acordo com o if
qtd_max.reduce(lambda el1, el2: el1 if el1[-1] > el2[-1] else el2)

[1752, 57, ['2018-10-24'], [4885], [27.35], 4885]

## Número de vendas no ano

In [16]:
qtd_vendida = dados_tratados.map(lambda el: len(el[-2]))

In [17]:
qtd_vendida.take(5)

[2, 1, 11, 3, 31]

In [18]:
qtd_vendida.reduce(lambda x,y: x + y)

18634896

## Quantidade vendida no ano

In [19]:
soma_qtd = dados_tratados.map(lambda el: sum(el[-2]))

In [20]:
soma_qtd.take(2)

[2, 1]

In [21]:
soma_qtd.reduce(lambda x, y: x + y)

37026070

## Receita total vendida no ano por loja

In [22]:
loja_receita = dados_tratados.map(lambda el: (el[1], sum(el[-1])))

In [23]:
loja_receita.take(5)

[(33, 185.14999999999998),
 (10, 99.29),
 (33, 1313.83),
 (10, 394.9),
 (1, 1432.2799999999997)]

In [24]:
grouped_loja_receita = loja_receita.groupByKey()

In [25]:
grouped_loja_receita.take(5)

[(14, <pyspark.resultiterable.ResultIterable at 0x7fbe5d468190>),
 (28, <pyspark.resultiterable.ResultIterable at 0x7fbe5d468650>),
 (42, <pyspark.resultiterable.ResultIterable at 0x7fbe5d468510>),
 (56, <pyspark.resultiterable.ResultIterable at 0x7fbe5d4681d0>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7fbe4060df10>)]

In [26]:
grouped_loja_receita2 = grouped_loja_receita.mapValues(lambda el: sum(el))

In [27]:
grouped_loja_receita2.count()

58

In [28]:
grouped_loja_receita2.collect()

[(14, 42843975.169999935),
 (28, 29700335.809999913),
 (42, 18952886.039999995),
 (56, 2808843.720000001),
 (1, 72432487.13000044),
 (15, 31231201.000000067),
 (29, 13114325.879999936),
 (43, 59514402.760000065),
 (57, 278447.64999999997),
 (2, 91234989.7600005),
 (16, 32085084.12000005),
 (30, 39147847.03),
 (44, 29391916.780000072),
 (58, 28025.269999999997),
 (3, 37149136.40000004),
 (17, 72383030.70999987),
 (31, 10394400.360000022),
 (45, 24940936.450000055),
 (4, 187991264.69999933),
 (18, 86107519.87000017),
 (32, 14901018.06999999),
 (46, 34809944.58999992),
 (33, 12904501.980000015),
 (5, 78133596.77999991),
 (19, 148188006.779999),
 (47, 9805724.529999984),
 (6, 23949395.88),
 (20, 39885542.16000008),
 (48, 46426019.55000003),
 (34, 8539140.499999998),
 (7, 37994650.91999993),
 (21, 99971539.41000007),
 (35, 11052525.090000013),
 (49, 21828545.940000147),
 (8, 45071710.85000019),
 (22, 73890631.34000008),
 (36, 15816487.329999996),
 (50, 19789046.430000074),
 (9, 32484778.159

# Exercícios 02
1. Qual é a loja com menor venda no ano?
2. Quantos produtos diferentes a loja 52 vendeu no ano?
3. Qual foi a venda (quantidade e receita) do produto 1752 no ano?

## Loja com menor venda no ano

In [29]:
loja_qtd_vendida = dados_tratados.map(lambda el: (el[1], len(el[-2])))

In [30]:
loja_qtd_vendida.take(5)

[(33, 2), (10, 1), (33, 11), (10, 3), (1, 31)]

In [31]:
grouped_loja_qtd = loja_qtd_vendida.groupByKey()

In [32]:
grouped_loja_qtd.take(2)

[(14, <pyspark.resultiterable.ResultIterable at 0x7fbe312eb190>),
 (28, <pyspark.resultiterable.ResultIterable at 0x7fbe312eb1d0>)]

In [33]:
grouped_loja_qtd2 = grouped_loja_qtd.mapValues(lambda el: sum(el))

In [34]:
grouped_loja_qtd2.takeOrdered(1, key = lambda x: x[-1])

[(58, 144)]

## Quantidade de produtos diferentes da loja 52

In [35]:
loja_52 = dados_tratados.filter(lambda x: x[1] == 52)

In [36]:
loja_52_prods = loja_52.map(lambda el: el[0])

In [37]:
loja_52_prods.distinct().count()

8861

## Venda (quantidade e receita) do produto 1752

In [38]:
prod_1752 = dados_tratados.filter(lambda x: x[0] == 1752)

In [39]:
prod_1752_qtd = prod_1752.map(lambda el: sum(el[-2]))

In [40]:
prod_1752_qtd.take(5)

[119, 461, 129, 340, 231]

In [41]:
prod_1752_qtd.reduce(lambda x, y: x + y)

12181

In [42]:
prod_1752_rec = prod_1752.map(lambda el: sum(el[-1]))

In [43]:
prod_1752_rec.take(5)

[3391.4400000000014,
 12983.560000000009,
 4240.060000000001,
 10077.760000000004,
 7668.919999999999]

In [44]:
prod_1752_rec.reduce(lambda x, y: x + y)

214743.9500000001

# Exercícios 03
1. Qual é a média e o desvio padrão de vendas por produto?
2. Quais são os 5 produtos com maior venda média?
3. Quais são os 5 produtos com venda média menor?

## Média e desvio padrão de vendas por produto

### Média

In [45]:
prod_vendas = dados_tratados.map(lambda el: [el[0], [sum(el[-2]), len(el[-2])]])

In [46]:
prod_vendas.take(5)

[[1, [2, 2]], [2, [1, 1]], [2, [18, 11]], [3, [3, 3]], [4, [49, 31]]]

In [47]:
grouped_prod_vendas = prod_vendas.groupByKey()

In [48]:
for t in grouped_prod_vendas.collect():
    print(t[0], [v for v in t[1]])

14 [[1, 1]]
28 [[200, 114], [93, 54], [272, 147], [464, 211], [484, 204], [76, 53], [87, 60], [65, 47], [137, 83], [80, 49], [132, 84], [36, 27], [117, 81], [42, 32], [97, 66], [34, 20], [282, 154], [114, 76], [265, 146], [267, 148], [58, 44], [184, 109], [53, 30], [77, 54], [31, 25], [172, 107], [35, 27], [65, 36], [54, 36], [186, 119], [17, 14], [22, 13], [21, 20], [37, 27], [62, 43], [66, 40], [45, 26], [105, 61], [19, 11], [19, 14], [29, 20], [57, 37], [90, 62], [47, 37], [95, 69], [14, 7], [76, 43], [38, 35], [129, 83], [62, 47], [57, 38], [47, 35], [100, 64], [2, 2]]
42 [[56, 43], [212, 134], [93, 74], [182, 122], [171, 105], [95, 73], [54, 45], [52, 44], [22, 20], [52, 41], [58, 44], [29, 21], [55, 42], [53, 39], [53, 41], [39, 30], [254, 155], [152, 103], [273, 160], [125, 78], [191, 120], [237, 154], [172, 119], [19, 18], [171, 120], [50, 37], [78, 55], [6, 2], [84, 66], [9, 7], [21, 20], [14, 10], [11, 6], [102, 75], [9, 7], [56, 42], [18, 15], [75, 53], [37, 24], [104, 80], 

11956 [[26, 15], [44, 26], [46, 38], [22, 19], [5, 4], [10, 10], [35, 20], [46, 28], [34, 26], [18, 14], [55, 40], [13, 10], [11, 8], [12, 9], [33, 20], [8, 7], [147, 96], [53, 41], [177, 110], [18, 13], [6, 5], [29, 28], [24, 18], [147, 82], [70, 42], [19, 18], [27, 20], [53, 30], [4, 4], [10, 6], [3, 3], [18, 15], [3, 3], [4, 3], [21, 16], [17, 17], [4, 3], [32, 28], [8, 8], [32, 21], [21, 15], [22, 13], [16, 12], [59, 32], [13, 11], [45, 34], [9, 8], [36, 26], [12, 8], [16, 13], [48, 34], [6, 6], [2, 1]]
11970 [[44, 30], [17, 15], [24, 22], [61, 40], [88, 63], [16, 12], [48, 28], [59, 41], [41, 30], [12, 10], [34, 27], [15, 10], [14, 13], [26, 16], [26, 21], [103, 66], [58, 51], [21, 15], [46, 33], [18, 16], [107, 71], [4, 2], [7, 7], [38, 29], [5, 5], [1, 1], [66, 48], [14, 7], [10, 10], [15, 15], [4, 3], [39, 32], [76, 58], [8, 7], [35, 23], [12, 10], [60, 44], [15, 13], [9, 9], [26, 22], [13, 10], [21, 16], [38, 25]]
11984 [[56, 46], [81, 41], [20, 15], [135, 87], [37, 32], [43, 

10151 [[2, 2], [3, 2]]
10165 [[31, 25], [210, 138], [31, 22], [118, 69], [68, 52], [19, 16], [51, 32], [18, 16], [27, 18], [11, 10], [41, 36], [11, 8], [46, 40], [8, 6], [18, 9], [60, 40], [63, 43], [89, 70], [9, 8], [37, 29], [57, 32], [6, 6], [38, 27], [48, 36], [18, 13], [11, 9], [8, 7], [30, 23], [58, 39], [48, 32], [2, 2], [14, 10], [15, 9], [1, 1], [36, 23], [2, 2], [21, 19], [9, 8], [7, 6], [29, 18], [28, 23], [33, 19], [5, 3], [23, 20], [17, 14], [15, 7], [3, 3], [3, 3]]
10179 [[29, 23], [22, 20], [12, 9], [12, 11], [32, 23], [3, 3], [41, 28], [6, 2], [5, 5], [24, 19], [8, 5], [14, 10], [5, 4], [12, 11], [23, 15], [52, 21], [96, 69], [137, 86], [45, 32], [3, 3], [21, 18], [2, 2], [1, 1], [2, 2], [9, 6], [3, 3], [9, 9], [3, 3], [2, 2], [4, 3], [7, 7], [5, 4], [2, 2], [1, 1]]
10193 [[532, 109], [449, 121], [299, 75], [521, 110], [572, 120], [45, 17], [282, 67], [136, 32], [361, 66], [453, 96], [244, 75], [135, 33], [72, 25], [282, 62], [97, 45], [168, 42], [195, 70], [885, 208], 

8346 [[1, 1]]
8360 [[518, 215], [196, 121], [543, 242], [654, 251], [609, 251], [77, 54], [237, 147], [793, 287], [246, 136], [408, 216], [292, 148], [440, 222], [145, 95], [177, 110], [272, 162], [180, 117], [345, 179], [308, 158], [782, 283], [385, 196], [720, 265], [404, 203], [178, 108], [234, 134], [134, 81], [899, 289], [254, 150], [477, 227], [436, 201], [321, 171], [200, 123], [427, 193], [213, 125], [220, 127], [442, 207], [286, 129], [221, 124], [199, 116], [144, 95], [75, 59], [349, 184], [406, 207], [132, 81], [317, 162], [89, 56], [51, 37], [565, 231], [115, 75], [593, 251], [216, 130], [137, 80], [341, 166], [260, 162], [24, 15]]
8374 [[51, 11], [72, 23], [2, 2], [43, 11], [49, 11], [10, 6], [1, 1], [102, 17], [26, 7], [1, 1], [8, 5], [39, 16], [104, 23], [3, 3], [58, 22], [3, 3], [23, 3], [9, 4], [186, 31], [165, 28], [76, 25], [47, 10], [6, 4], [4, 2], [20, 11], [66, 14], [58, 15], [105, 18], [46, 25], [30, 16], [82, 19], [12, 5], [63, 20], [23, 10], [26, 10], [47, 14],

6583 [[6, 5], [18, 12], [29, 24], [3, 3], [2, 2], [2, 2], [2, 2], [10, 5], [2, 2], [3, 3], [2, 2], [3, 3], [2, 2], [15, 13], [1, 1], [19, 17], [3, 3], [7, 7], [1, 1], [7, 5], [2, 2], [2, 2], [2, 2], [1, 1], [7, 4], [3, 3], [3, 3], [12, 9], [13, 5], [1, 1], [2, 2], [12, 6], [1, 1]]
6597 [[1, 1], [26, 11], [65, 29], [56, 21], [174, 57], [39, 11], [32, 20], [24, 12], [82, 45], [67, 34], [5, 2], [3, 3], [50, 20], [260, 109], [57, 35], [27, 10], [28, 20], [91, 44], [33, 15], [28, 23], [30, 15], [21, 4], [50, 27], [28, 18], [11, 8], [24, 8], [34, 9], [330, 146], [79, 12], [13, 5], [40, 19], [132, 65], [12, 7], [17, 15], [55, 13], [13, 8], [62, 18], [44, 30], [20, 18], [126, 51], [52, 22], [31, 18], [28, 17], [65, 24], [65, 23], [9, 7]]
6611 [[16, 9], [2, 2]]
6625 [[10, 7], [22, 20], [3, 3], [58, 34], [8, 7], [16, 11], [13, 7], [1, 1], [3, 3], [4, 4], [2, 2], [1, 1], [22, 18], [9, 6], [39, 33], [93, 52], [1, 1], [10, 8], [17, 14], [3, 3], [3, 3], [7, 7], [1, 1], [4, 3], [13, 11], [7, 4], [3, 

...

3379 [[90, 66], [99, 70], [93, 64], [300, 164], [164, 98], [33, 26], [28, 25], [33, 28], [41, 29], [50, 38], [132, 82], [18, 13], [22, 11], [10, 8], [12, 10], [41, 24], [94, 61], [89, 40], [157, 91], [62, 46], [60, 38], [30, 17], [16, 15], [20, 16], [22, 14], [64, 43], [41, 17], [5, 5], [29, 22], [52, 33], [16, 14], [6, 6], [2, 1], [49, 41], [49, 29], [43, 25], [20, 8], [20, 15], [33, 12], [22, 12], [53, 32], [47, 32], [6, 5], [31, 27], [84, 50], [24, 11], [10, 8], [31, 27], [58, 37], [56, 29], [34, 22], [3, 2]]
3393 [[71, 48], [79, 58], [14, 13], [97, 68], [109, 79], [9, 9], [44, 37], [19, 15], [13, 8], [27, 15], [80, 56], [54, 43], [71, 47], [30, 25], [18, 13], [16, 12], [101, 75], [59, 43], [171, 118], [33, 25], [125, 92], [74, 53], [33, 30], [1, 1], [49, 39], [35, 26], [10, 7], [32, 20], [93, 71], [11, 7], [52, 43], [61, 50], [14, 12], [23, 22], [49, 27], [38, 30], [52, 37], [39, 29], [34, 23], [21, 15], [29, 20], [14, 13], [52, 42], [43, 35], [2, 2], [38, 31], [53, 42], [40, 34], 

1476 [[358, 181], [281, 166], [99, 65], [395, 217], [155, 106], [126, 90], [69, 51], [43, 34], [128, 88], [136, 96], [350, 201], [124, 94], [74, 55], [59, 49], [120, 92], [24, 17], [355, 212], [291, 165], [407, 213], [105, 78], [234, 145], [90, 70], [97, 69], [96, 78], [201, 146], [136, 97], [63, 52], [57, 42], [110, 80], [197, 132], [66, 53], [107, 77], [7, 6], [19, 14], [97, 75], [150, 104], [60, 42], [42, 33], [136, 97], [76, 59], [57, 42], [54, 40], [102, 66], [23, 20], [332, 189], [20, 15], [35, 28], [180, 121], [107, 80], [163, 115], [45, 38], [23, 18], [392, 219], [79, 61], [11, 8]]
1490 [[5, 5], [3, 3], [2, 2], [25, 16], [4, 2], [4, 4], [1, 1], [16, 3], [17, 14], [11, 7], [13, 7], [21, 10], [3, 1], [22, 14], [11, 6]]
1504 [[2, 1], [1, 1]]
1518 [[5, 4], [2, 2], [76, 54], [45, 34], [23, 18], [2, 2], [1, 1], [4, 3], [3, 3], [13, 10], [9, 9], [9, 9], [8, 6], [53, 34], [2, 1], [1, 1], [15, 14], [16, 10], [34, 27], [38, 25], [7, 6], [76, 38], [58, 44], [66, 42], [2, 2]]
1532 [[2, 2]]

13894 [[17, 9], [37, 31], [7, 3], [4, 3], [3, 3], [2, 2], [18, 14], [7, 6], [14, 10], [16, 16], [3, 2], [5, 5], [3, 3], [27, 7], [3, 3], [3, 3], [1, 1], [2, 2], [4, 2], [5, 3], [1, 1]]
13908 [[1, 1], [1, 1]]
13922 [[7, 5], [13, 9], [13, 8], [4, 3], [3, 2], [2, 2], [5, 3]]
7 [[494, 195], [1039, 290], [334, 152], [379, 139], [236, 103], [187, 76], [239, 93], [468, 180], [30, 23], [200, 88], [366, 151], [556, 212], [119, 53], [200, 101], [168, 76], [342, 136], [276, 143], [699, 240], [320, 134], [387, 165], [742, 242], [210, 91], [159, 82], [540, 168], [75, 45], [663, 223], [98, 54], [131, 58], [117, 59], [143, 60], [147, 77], [102, 42], [126, 62], [193, 91], [53, 27], [189, 86], [154, 75], [204, 99], [386, 147], [376, 157], [331, 116], [697, 253], [139, 67], [147, 73], [405, 156], [62, 31], [412, 196], [109, 48], [205, 91], [262, 113], [170, 88], [293, 140], [470, 178], [25, 13], [5, 2]]
21 [[141, 99], [46, 33], [114, 75], [284, 180], [141, 95], [5, 5], [51, 38], [86, 62], [93, 70], [34,

12285 [[41, 19], [72, 42], [67, 43], [78, 45], [87, 51], [18, 15], [82, 45], [19, 14], [23, 10], [43, 22], [36, 17], [31, 19], [16, 11], [35, 15], [43, 29], [52, 25], [17, 11], [63, 40], [57, 33], [11, 7], [103, 66], [18, 15], [23, 19], [26, 11], [29, 15], [72, 46], [22, 12], [37, 29], [12, 9], [52, 26], [6, 6], [10, 8], [1, 1], [30, 18], [4, 3], [24, 17], [75, 39], [28, 22], [60, 37], [10, 8], [35, 27], [9, 4], [51, 34], [21, 20], [23, 15], [34, 31], [32, 18], [49, 37], [16, 13], [14, 11], [52, 34], [21, 15], [56, 35], [43, 22], [11, 7]]
12299 [[55, 35], [32, 24], [23, 13], [106, 53], [43, 29], [2, 2], [20, 13], [7, 4], [5, 3], [60, 29], [25, 18], [52, 34], [17, 10], [15, 7], [38, 26], [7, 6], [34, 20], [91, 52], [99, 55], [30, 21], [58, 38], [12, 10], [75, 51], [27, 19], [5, 5], [23, 17], [20, 16], [19, 12], [3, 3], [3, 3], [14, 3], [8, 7], [5, 2], [4, 4], [13, 13], [4, 4], [198, 109], [19, 14], [39, 27], [5, 3], [11, 5], [10, 7], [35, 18], [34, 18], [14, 7], [89, 45], [42, 14], [17,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [49]:
grouped_prod_vendas.take(5)

[(14, <pyspark.resultiterable.ResultIterable at 0x7fbe28e8ed10>),
 (28, <pyspark.resultiterable.ResultIterable at 0x7fbe28e8ef10>),
 (42, <pyspark.resultiterable.ResultIterable at 0x7fbe28e8e210>),
 (56, <pyspark.resultiterable.ResultIterable at 0x7fbe28e8e810>),
 (70, <pyspark.resultiterable.ResultIterable at 0x7fbe28e8e550>)]

In [50]:
def count_prod (el):
    
    total_prod = sum(pair[0] for pair in el)
    total_sell = sum(pair[1] for pair in el)
    
    return [total_prod,total_sell]

In [51]:
grouped_prod_vendas2 = grouped_prod_vendas.mapValues(count_prod)

In [52]:
grouped_prod_vendas2.take(30)

[(14, [1, 1]),
 (28, [5515, 3291]),
 (42, [4195, 2927]),
 (56, [4490, 3027]),
 (70, [3, 1]),
 (84, [4542, 1493]),
 (98, [7075, 4179]),
 (112, [7, 7]),
 (126, [1, 1]),
 (140, [1288, 968]),
 (154, [3251, 2346]),
 (168, [2494, 1118]),
 (182, [6345, 3993]),
 (196, [16379, 8540]),
 (210, [13134, 3481]),
 (224, [8664, 1827]),
 (238, [20, 17]),
 (252, [246, 177]),
 (266, [1556, 740]),
 (280, [272, 206]),
 (294, [5555, 3492]),
 (308, [1907, 1323]),
 (322, [1518, 673]),
 (336, [158, 114]),
 (350, [4211, 2853]),
 (364, [15244, 7455]),
 (378, [6037, 3379]),
 (392, [11497, 3552]),
 (406, [1211, 914]),
 (420, [190, 79])]

In [53]:
media_prod_vendas = grouped_prod_vendas2.mapValues(lambda el: el[0] / el[1])

In [54]:
media_prod_vendas.count()

13923

In [55]:
media_prod_vendas.take(30)

[(14, 1.0),
 (28, 1.6757824369492555),
 (42, 1.4332080628629997),
 (56, 1.4833168153287082),
 (70, 3.0),
 (84, 3.042196918955124),
 (98, 1.6929887532902608),
 (112, 1.0),
 (126, 1.0),
 (140, 1.3305785123966942),
 (154, 1.385763000852515),
 (168, 2.230769230769231),
 (182, 1.589030803906837),
 (196, 1.9179156908665105),
 (210, 3.773053720195346),
 (224, 4.742200328407225),
 (238, 1.1764705882352942),
 (252, 1.3898305084745763),
 (266, 2.1027027027027025),
 (280, 1.3203883495145632),
 (294, 1.59077892325315),
 (308, 1.4414210128495843),
 (322, 2.2555720653789004),
 (336, 1.3859649122807018),
 (350, 1.4759901857693656),
 (364, 2.0448021462105967),
 (378, 1.786623261319917),
 (392, 3.236768018018018),
 (406, 1.324945295404814),
 (420, 2.4050632911392404)]

### Desvio Padrão

In [70]:
dp_prod_vendas = grouped_prod_vendas2.leftOuterJoin(media_prod_vendas)

In [71]:
dp_prod_vendas.take(5)

[(14, ([1, 1], 1.0)),
 (28, ([5515, 3291], 1.6757824369492555)),
 (42, ([4195, 2927], 1.4332080628629997)),
 (56, ([4490, 3027], 1.4833168153287082)),
 (70, ([3, 1], 3.0))]

In [75]:
dp_prod_vendas2 = dp_prod_vendas.map(lambda x: [x[0], x[1][0][0], x[1][0][1], x[1][1]])

In [76]:
dp_prod_vendas2.take(5)

[[14, 1, 1, 1.0],
 [28, 5515, 3291, 1.6757824369492555],
 [42, 4195, 2927, 1.4332080628629997],
 [56, 4490, 3027, 1.4833168153287082],
 [70, 3, 1, 3.0]]

In [79]:
dp_prod_vendas3 = dp_prod_vendas2.map(lambda x: [x[0], ((x[1]-x[3])/x[2])**0.5])

In [80]:
dp_prod_vendas3.take(30)

[[14, 0.0],
 [28, 1.2943234662780247],
 [42, 1.1969621598047653],
 [56, 1.217713753986703],
 [70, 0.0],
 [84, 1.7436052531143358],
 [98, 1.3009933263238511],
 [112, 0.9258200997725514],
 [126, 0.0],
 [140, 1.1529110754163878],
 [154, 1.1769334344117746],
 [168, 1.4929078703837337],
 [182, 1.2604097943867896],
 [196, 1.3848072467223878],
 [210, 1.9421559723208501],
 [224, 2.1770633217686783],
 [238, 1.0522672835293128],
 [252, 1.1755757562612386],
 [266, 1.4490897876128708],
 [280, 1.1462890984967333],
 [294, 1.2610802407988628],
 [308, 1.2001381184645574],
 [322, 1.5007399997028812],
 [336, 1.1720952714395152],
 [350, 1.214690429307651],
 [364, 1.4298698752713208],
 [378, 1.3364484719297316],
 [392, 1.7988487334294379],
 [406, 1.1504328244118558],
 [420, 1.5409800297248812]]

## 5 produtos com maior venda média

In [81]:
media_prod_vendas.takeOrdered(5, lambda el: -el[-1])

[(1908, 16.165745856353592),
 (416, 15.333333333333334),
 (66, 14.538029386343993),
 (68, 14.487804878048781),
 (3475, 14.026223776223777)]

## 5 produtos com menor venda média

In [82]:
media_prod_vendas.takeOrdered(5, lambda el: el[-1])

[(14, 1.0), (112, 1.0), (126, 1.0), (448, 1.0), (630, 1.0)]

# Exercícios 04
1. Encontre os produtos que venderam em todas as lojas no ano;
2. Existe algum produto que não vendeu na loja 33 mas vendeu na 10? Quais?

## Produtos que venderam em todas as lojas no ano

In [58]:
prod_lojas = dados_tratados.map(lambda el: (el[0], el[1]))

In [59]:
grouped_prod_lojas = prod_lojas.groupByKey()

In [60]:
for t in grouped_prod_lojas.take(5):
    print(t[0], [v for v in t[1]])

14 [33]
28 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
42 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
56 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
70 [27]


In [61]:
count_prod_lojas = grouped_prod_lojas.mapValues(lambda el: len(el))

In [62]:
count_prod_lojas.take(10)

[(14, 1),
 (28, 54),
 (42, 52),
 (56, 54),
 (70, 1),
 (84, 52),
 (98, 55),
 (112, 1),
 (126, 1),
 (140, 43)]

In [63]:
qtd_lojas_58 = count_prod_lojas.filter(lambda x: x[1] == 58)

In [64]:
qtd_lojas_58.collect()

[]

## Produtos que venderam na loja 10, mas não na loja 33

In [94]:
prod_lojas.take(5)

[(1, 33), (2, 10), (2, 33), (3, 10), (4, 1)]

In [97]:
prods_loja10 = prod_lojas.filter(lambda x: x[1] == 10)

In [98]:
prods_loja10.take(5)

[(2, 10), (3, 10), (4, 10), (5, 10), (6, 10)]

In [99]:
prods_loja33 = prod_lojas.filter(lambda x: x[1] == 33)

In [100]:
prods_loja33.take(5)

[(1, 33), (2, 33), (4, 33), (5, 33), (6, 33)]

In [103]:
dif_10_33 = prods_loja10.subtract(prods_loja33).distinct()

In [104]:
dif_10_33.count()

8909

In [105]:
dif_10_33.collect()

[(16, 10),
 (84, 10),
 (104, 10),
 (108, 10),
 (136, 10),
 (140, 10),
 (160, 10),
 (228, 10),
 (248, 10),
 (260, 10),
 (284, 10),
 (304, 10),
 (372, 10),
 (532, 10),
 (552, 10),
 (556, 10),
 (696, 10),
 (720, 10),
 (840, 10),
 (844, 10),
 (896, 10),
 (964, 10),
 (984, 10),
 (1024, 10),
 (1112, 10),
 (1116, 10),
 (1136, 10),
 (1168, 10),
 (1256, 10),
 (1260, 10),
 (1288, 10),
 (1292, 10),
 (1312, 10),
 (1380, 10),
 (1404, 10),
 (1456, 10),
 (1540, 10),
 (1560, 10),
 (1564, 10),
 (1584, 10),
 (1684, 10),
 (1704, 10),
 (1708, 10),
 (1728, 10),
 (1852, 10),
 (1996, 10),
 (2052, 10),
 (2072, 10),
 (2076, 10),
 (2164, 10),
 (2196, 10),
 (2216, 10),
 (2220, 10),
 (2240, 10),
 (2340, 10),
 (2360, 10),
 (2364, 10),
 (2484, 10),
 (2504, 10),
 (2528, 10),
 (2632, 10),
 (2636, 10),
 (2656, 10),
 (2688, 10),
 (2776, 10),
 (2780, 10),
 (2800, 10),
 (2900, 10),
 (2920, 10),
 (2952, 10),
 (2956, 10),
 (2976, 10),
 (3044, 10),
 (3068, 10),
 (3080, 10),
 (3084, 10),
 (3104, 10),
 (3172, 10),
 (3192, 10)

# Tempo de execução

In [ ]:
# finaliza a sessão spark
#spark.stop()

In [ ]:
end = dt.datetime.now()

print('Tempo de execução: {}'.format(end - start))

Com 2 processadores: 0:01:24.175375

Com 4 processadores: 0:01:23.309407

Com 6 processadores: 0:01:23.908466